In [18]:
import openpyxl
from datetime import datetime

# 新規ワークブックを作成
wb = openpyxl.Workbook()
ws = wb.active
#今日の日付取得
today = datetime.today()
#請求書番号
id_num = 1

#ヘッダ部
ws['B2'].value = '請求書'
ws['B4'].value = '株式会社ABC'
ws['B5'].value = '〒101-0022 東京都千代田区神田練塀町300'
ws['B6'].value = 'TEL:03-1234-5678 FAX:03-1234-5678'
ws['B7'].value = '担当者名:鈴木一郎 様'

ws['F4'].value = 'No.'
ws['G4'].value = formatted_id = f"{id_num:04}"

ws['F5'].value = '日付'
ws['G5'].value = today.strftime('%Y/%m/%d')

#データ部
header = ['商品名', '数量', '単価', '金額']
start_row = 10
start_col = 2
for col_num, col_name in enumerate(header, start=start_col):
    ws.cell(row=start_row, column=col_num, value=col_name)

data = [
    ['商品A', 2, 10000],
    ['商品B', 1, 15000]
]
for row_num, item in enumerate(data, start=start_row + 1):
    for col_num, value in enumerate(item, start=start_col):
        ws.cell(row=row_num, column=col_num, value=value)

    # 売上の計算式をD列（4番目の列）に記入
    quantity_cell = ws.cell(row=row_num, column=start_col + 1).coordinate # 数量セル
    price_cell = ws.cell(row=row_num, column=start_col + 2).coordinate    # 単価セル
    sales_cell = ws.cell(row=row_num, column=start_col + 3)               # 売上セル
    sales_cell.value = f"={quantity_cell}*{price_cell}"                   # 計算式:数量*単価

#フッタ部
quantity_cell_s = ws.cell(row=11, column=5).coordinate               # 先頭行単価セル
quantity_cell_e = ws.cell(row=row_num, column=5).coordinate          # 最終行単価セル
total_cell1 = ws.cell(row=row_num + 1, column=5).coordinate          # 合計セル1
total_cell2 = ws.cell(row=row_num + 3, column=5).coordinate          # 合計セル2
tax_cell = ws.cell(row=row_num + 4, column=start_col + 3).coordinate # 消費税セル

ws.cell(row=row_num+3, column=2).value='合計'
ws.cell(row=row_num+4, column=2).value='消費税'
ws.cell(row=row_num+5, column=2).value='税込金額'
ws.cell(row=row_num+1, column=5).value=f"=SUM({quantity_cell_s}:{quantity_cell_e})" # 計算式:合計1
ws.cell(row=row_num+3, column=5).value=f"={total_cell1}"                            # 計算式:合計2
ws.cell(row=row_num+4, column=5).value=f"={total_cell2}*0.1"                        # 計算式:消費税
ws.cell(row=row_num+5, column=5).value=f"=SUM({total_cell2}:{tax_cell})"            # 計算式:税込金額

# 列幅の設定
ws.column_dimensions['E'].width = 10  # E列の幅を15に設定

# 保存
backup_filename = f"請求書_{today.strftime('%Y%m%d')}.xlsx"
wb.save(backup_filename)